# Análise de feedbacks

Análise dos comentários presentes nas base 'barber_shop'.

## Extração de dados

In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import os

load_dotenv()

True

In [2]:
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')
ssl_cert= '../configs/ca.crt'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}?sslmode=verify-full&sslrootcert={ssl_cert}')

In [3]:
with open('../sql_queries/select_feedbacks.sql', 'r') as file:
    query = file.read()

data = pd.read_sql(query, engine)

In [4]:
data

,dat_comp,av_comentario,nm_cliente,nm_funcionario,tp_servico
0,2022-02-01,"Resultado aceitável, mas esperava mais.",Catarina Rocha,Maria Oliveira,Alisamento
1,2022-04-01,"Não gostei, o profissional foi desatento e o c...",Anthony da Mata,João Silva,Alisamento
2,2022-03-01,"Péssimo serviço, não volto mais.",Kamilly Santos,Maria Oliveira,Alisamento
3,2021-12-01,"O resultado não foi satisfatório, poderia ser ...",Kamilly Gonçalves,Maria Oliveira,Alisamento
4,2021-07-01,"Foi bom, mas o atendimento precisa melhorar.",Heloísa Correia,Carlos Souza,Alisamento
...,...,...,...,...,...
2995,2023-06-01,"Não gostei muito, não ficou como esperado.",Enzo Gomes,Ana Lima,Tratamento capilar
2996,2023-08-01,"Corte bom, voltarei mais vezes.",Lucca Ferreira,João Silva,Tratamento capilar
2997,2022-06-01,"Corte horrível, totalmente diferente do que pedi.",Rafaela da Rosa,Ana Lima,Tratamento capilar
2998,2022-09-01,"Bom serviço, mas poderia ser um pouco mais ráp...",Helena Melo,Ana Lima,Tratamento capilar


# Pré-processamento do texto

Cada comentário define um corpus

## Tokenização

Tokenizando cada comentário individualmente

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
data['tokens'] = data['av_comentario'].apply(
    lambda comentario: nltk.word_tokenize(comentario.lower(), language= 'portuguese')
    )

### Remoção de stopwords

In [8]:
from nltk.corpus import stopwords

In [10]:
data['tokens_sem_stopwords'] = data['tokens'].apply(
    lambda tokens: [token for token in tokens if token not in stopwords.words('portuguese')]
)

### Remoção de números

In [11]:
import re

In [12]:
data['tokens_sem_stopwords_sem_numeros'] = data['tokens_sem_stopwords'].apply(
    lambda tokens: [re.sub('\d+', '', token) for token in tokens]
)

### Remoção de pontuação

In [13]:
import string

In [14]:
data['tokens_sem_stopwords_sem_numeros_sem_pontuacao'] = data['tokens_sem_stopwords_sem_numeros'].apply(
    lambda tokens: [token for token in tokens if token not in string.punctuation]
)

In [15]:
data

,dat_comp,av_comentario,nm_cliente,nm_funcionario,tp_servico,tokens,tokens_sem_stopwords,tokens_sem_stopwords_sem_numeros,tokens_sem_stopwords_sem_numeros_sem_pontuacao
0,2022-02-01,"Resultado aceitável, mas esperava mais.",Catarina Rocha,Maria Oliveira,Alisamento,"[resultado, aceitável, ,, mas, esperava, mais, .]","[resultado, aceitável, ,, esperava, .]","[resultado, aceitável, ,, esperava, .]","[resultado, aceitável, esperava]"
1,2022-04-01,"Não gostei, o profissional foi desatento e o c...",Anthony da Mata,João Silva,Alisamento,"[não, gostei, ,, o, profissional, foi, desaten...","[gostei, ,, profissional, desatento, corte, fi...","[gostei, ,, profissional, desatento, corte, fi...","[gostei, profissional, desatento, corte, ficou..."
2,2022-03-01,"Péssimo serviço, não volto mais.",Kamilly Santos,Maria Oliveira,Alisamento,"[péssimo, serviço, ,, não, volto, mais, .]","[péssimo, serviço, ,, volto, .]","[péssimo, serviço, ,, volto, .]","[péssimo, serviço, volto]"
3,2021-12-01,"O resultado não foi satisfatório, poderia ser ...",Kamilly Gonçalves,Maria Oliveira,Alisamento,"[o, resultado, não, foi, satisfatório, ,, pode...","[resultado, satisfatório, ,, poderia, melhor, .]","[resultado, satisfatório, ,, poderia, melhor, .]","[resultado, satisfatório, poderia, melhor]"
4,2021-07-01,"Foi bom, mas o atendimento precisa melhorar.",Heloísa Correia,Carlos Souza,Alisamento,"[foi, bom, ,, mas, o, atendimento, precisa, me...","[bom, ,, atendimento, precisa, melhorar, .]","[bom, ,, atendimento, precisa, melhorar, .]","[bom, atendimento, precisa, melhorar]"
...,...,...,...,...,...,...,...,...,...
2995,2023-06-01,"Não gostei muito, não ficou como esperado.",Enzo Gomes,Ana Lima,Tratamento capilar,"[não, gostei, muito, ,, não, ficou, como, espe...","[gostei, ,, ficou, esperado, .]","[gostei, ,, ficou, esperado, .]","[gostei, ficou, esperado]"
2996,2023-08-01,"Corte bom, voltarei mais vezes.",Lucca Ferreira,João Silva,Tratamento capilar,"[corte, bom, ,, voltarei, mais, vezes, .]","[corte, bom, ,, voltarei, vezes, .]","[corte, bom, ,, voltarei, vezes, .]","[corte, bom, voltarei, vezes]"
2997,2022-06-01,"Corte horrível, totalmente diferente do que pedi.",Rafaela da Rosa,Ana Lima,Tratamento capilar,"[corte, horrível, ,, totalmente, diferente, do...","[corte, horrível, ,, totalmente, diferente, pe...","[corte, horrível, ,, totalmente, diferente, pe...","[corte, horrível, totalmente, diferente, pedi]"
2998,2022-09-01,"Bom serviço, mas poderia ser um pouco mais ráp...",Helena Melo,Ana Lima,Tratamento capilar,"[bom, serviço, ,, mas, poderia, ser, um, pouco...","[bom, serviço, ,, poderia, pouco, rápido, .]","[bom, serviço, ,, poderia, pouco, rápido, .]","[bom, serviço, poderia, pouco, rápido]"
